## testing cnn

# inputs
- grid

In [12]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization


In [6]:

# Load the datasets
player_positions = pd.read_csv('data/train_locs.csv')
game_outcomes = pd.read_csv('data/train_pbp.csv')

# Parameters
court_length, court_width = 94, 50
grid_size = 1  # 1 foot grid size
n_classes = 2  # Offensive rebound or not

# Convert court dimensions to a grid
n_rows, n_cols = int(court_length / grid_size), int(court_width / grid_size)

def positions_to_grid(data, n_rows, n_cols):
    # Initialize an empty grid
    grid = np.zeros((n_rows, n_cols, 1))  # Single channel for player positions
    
    # Map player positions to grid
    for _, row in data.iterrows():
        x, y, code = min(int(row['court_x'] // grid_size), n_rows - 1), min(int(row['court_y'] // grid_size), n_cols - 1), row['annotation_code']
        if 'd' in code:  # Defense
            grid[x, y, 0] = 1
        elif 't' in code:  # Offense excluding shooter
            grid[x, y, 0] = 2
        elif 's' in code:  # Shooter
            grid[x, y, 0] = 3
    
    return grid

# Preprocess the data
unique_ids = player_positions['id'].unique()
X = np.array([positions_to_grid(player_positions[player_positions['id'] == uid], n_rows, n_cols) for uid in unique_ids])

# Prepare labels
y = game_outcomes.set_index('id').loc[unique_ids]['is_oreb'].values
y = to_categorical(y, num_classes=n_classes)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Multiply, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model

# Input shape based on the court grid and assuming a single channel for simplicity
input_shape = (n_rows, n_cols, 1)  # n_rows=94, n_cols=50, channels=1 for player positions

# Model architecture
inputs = Input(shape=input_shape)

# Base convolutional layers with fewer neurons
conv1 = Conv2D(16, kernel_size=(3, 3), activation='relu')(inputs)  # Reduced from 32 to 16
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(32, (3, 3), activation='relu')(pool1)  # Reduced from 64 to 32
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

# Spatial attention layer remains the same
attention_probs = Conv2D(1, kernel_size=(1, 1), activation='sigmoid')(pool2)
attention_mul = Multiply()([pool2, attention_probs])

# Flattening and dense layers with fewer neurons
flatten = Flatten()(attention_mul)
dense1 = Dense(64, activation='relu')(flatten)  # Reduced from 128 to 64
dropout = Dropout(0.5)(dense1)
output = Dense(2, activation='softmax')(dropout)  # Output layer remains the same for binary classification

# Construct and compile the model
model = Model(inputs=[inputs], outputs=[output])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary to inspect the architecture
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 94, 50, 1)]          0         []                            
                                                                                                  
 conv2d_9 (Conv2D)           (None, 92, 48, 16)           160       ['input_4[0][0]']             
                                                                                                  
 max_pooling2d_6 (MaxPoolin  (None, 46, 24, 16)           0         ['conv2d_9[0][0]']            
 g2D)                                                                                             
                                                                                                  
 conv2d_10 (Conv2D)          (None, 44, 22, 32)           4640      ['max_pooling2d_6[0][0]'

In [20]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=10,  # Adjust based on convergence and computational resources
                    validation_data=(X_test, y_test))

Epoch 1/10
771/771 [==============================] - 9s 10ms/step - loss: 0.5937 - accuracy: 0.7177 - val_loss: 0.5930 - val_accuracy: 0.7174
Epoch 2/10
771/771 [==============================] - 8s 10ms/step - loss: 0.5862 - accuracy: 0.7182 - val_loss: 0.5906 - val_accuracy: 0.7174
Epoch 3/10
771/771 [==============================] - 8s 10ms/step - loss: 0.5818 - accuracy: 0.7184 - val_loss: 0.5936 - val_accuracy: 0.7174
Epoch 4/10
771/771 [==============================] - 8s 11ms/step - loss: 0.5744 - accuracy: 0.7186 - val_loss: 0.5923 - val_accuracy: 0.7176
Epoch 5/10
771/771 [==============================] - 9s 12ms/step - loss: 0.5659 - accuracy: 0.7222 - val_loss: 0.5961 - val_accuracy: 0.7156
Epoch 6/10
771/771 [==============================] - 8s 11ms/step - loss: 0.5499 - accuracy: 0.7318 - val_loss: 0.6024 - val_accuracy: 0.7153
Epoch 7/10
771/771 [==============================] - 9s 12ms/step - loss: 0.5313 - accuracy: 0.7447 - val_loss: 0.6148 - val_accuracy: 0.7038

In [ ]:
import matplotlib.pyplot as plt

# Plotting training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plotting training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()